In [1]:
! pip install -q --user google-cloud-aiplatform google-cloud-discoveryengine langchain-google-vertexai langchain-google-community

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cloud-tpu-client 0.10 requires google-api-python-client==1.8.0, but you have google-api-python-client 2.159.0 which is incompatible.
kfp 2.5.0 requires requests-toolbelt<1,>=0.8.0, but you have requests-toolbelt 1.0.0 which is incompatible.


In [2]:
# Restart kernel after packages are installed so that your environment can access the new packages
import IPython
import time

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
PROJECT_ID = "qwiklabs-gcp-00-e53e20d40da1"
LOCATION = "us-central1"

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [2]:
####Task 3. Import Datastore, Model information and Libraries.

In [3]:
DATA_STORE_ID = "qna-datastore-id"  # @param {type:"string"}
DATA_STORE_LOCATION = "global"  # @param {type:"string"}

MODEL = "gemini-1.0-pro"  # @param {type:"string"}

if PROJECT_ID == "YOUR_PROJECT_ID" or DATA_STORE_ID == "YOUR_DATA_STORE_ID":
    raise ValueError(
        "Please set the PROJECT_ID, DATA_STORE_ID constants to reflect your environment."
    )

In [4]:
from langchain.chains import RetrievalQA
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

from langchain_google_vertexai import VertexAI
from langchain_google_community import VertexAISearchRetriever
from langchain_google_community import VertexAIMultiTurnSearchRetriever

In [8]:
"""
Task 4. LangChain retrieval Q&A chains
Here, we'll be having three LangChain retrieval Q&A chains:

RetrivalQA
RetrievalQAWithSourceChain
ConversationalRetrivealChain
We begin by initializing a Vertex AI LLM and a LangChain 'retriever' to fetch documents from our Vertex AI Search engine.
"""

"\nTask 4. LangChain retrieval Q&A chains\nHere, we'll be having three LangChain retrieval Q&A chains:\n\nRetrivalQA\nRetrievalQAWithSourceChain\nConversationalRetrivealChain\nWe begin by initializing a Vertex AI LLM and a LangChain 'retriever' to fetch documents from our Vertex AI Search engine.\n"

In [9]:
llm = VertexAI(model_name=MODEL)

retriever = VertexAISearchRetriever(
    project_id=PROJECT_ID,
    location_id=DATA_STORE_LOCATION,
    data_store_id=DATA_STORE_ID,
    get_extractive_answers=True,
    max_documents=10,
    max_extractive_segment_count=1,
    max_extractive_answer_count=5,
)

/home/jupyter/.local/lib/python3.10/site-packages/langchain_google_community/vertex_ai_search.py:364: UserWarning: Beta features are configured but beta=False. The following beta features will be ignored:['custom_embedding_ratio']
  warnings.warn(


In [10]:
search_query = "What was Alphabet's Revenue in Q2 2021?"  # @param {type:"string"}

retrieval_qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=retriever
)
retrieval_qa.invoke(search_query)

{'query': "What was Alphabet's Revenue in Q2 2021?",
 'result': "Alphabet's revenue in Q2 2021 was $65.1 billion. \n"}

In [11]:
retrieval_qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True
)

results = retrieval_qa.invoke(search_query)

print("*" * 79)
print(results["result"])
print("*" * 79)
for doc in results["source_documents"]:
    print("-" * 79)
    print(doc.page_content)

*******************************************************************************
## Alphabet's Revenue in Q2 2021: 

Alphabet's Q2 2021 revenue was **$61.88 billion**. This information is found in the second sentence of the first paragraph of the document you provided:

> "Our strong second quarter revenues of **$61.9 billion** reflect elevated consumer online activity and broad-based strength in advertiser spend." 

It is important to note that the document also mentions a **$69.7 billion** revenue figure for Q2 2022, which is significantly higher than the $61.88 billion for Q2 2021. 

*******************************************************************************
-------------------------------------------------------------------------------
Our long-term investments in AI and Google Cloud are helping us drive significant improvements in everyone&#39;s digital experience.” “Our strong second quarter revenues of <b>$61.9 billion</b> reflect elevated consumer online activity and broad-b

In [12]:
retrieval_qa_with_sources = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm, chain_type="stuff", retriever=retriever
)

retrieval_qa_with_sources.invoke(search_query, return_only_outputs=True)

{'answer': "Alphabet's Revenue in Q2 2021 was $61.88 billion\n",
 'sources': 'gs://cloud-samples-data/gen-app-builder/search/alphabet-investor-pdfs/2021Q2_alphabet_earnings_release.pdf1, gs://cloud-samples-data/gen-app-builder/search/alphabet-investor-pdfs/2021Q2_alphabet_earnings_release.pdf5.'}

In [13]:
multi_turn_retriever = VertexAIMultiTurnSearchRetriever(
    project_id=PROJECT_ID, location_id=DATA_STORE_LOCATION, data_store_id=DATA_STORE_ID
)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
conversational_retrieval = ConversationalRetrievalChain.from_llm(
    llm=llm, retriever=multi_turn_retriever, memory=memory
)

search_query = "What were alphabet revenues in 2022?"

result = conversational_retrieval.invoke(search_query)
print(result["answer"])

/var/tmp/ipykernel_3868/1182819151.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


Based on the provided context, Alphabet's 2022 revenues were $282,836 million. This information can be found on page 6 of the document, under the section titled "Alphabet Inc. CONSOLIDATED STATEMENTS OF INCOME." 

It is important to note that the document also provides information about Alphabet's revenues in other years, as well as other financial information such as costs and expenses, income, and earnings per share. 



In [14]:
new_query = "What about costs and expenses?"
result = conversational_retrieval.invoke(new_query)
print(result["answer"])

In 2022, Alphabet's total costs and expenses were $207,994 million. 


In [15]:
new_query = "Is this more than in 2021?"

result = conversational_retrieval.invoke(new_query)
print(result["answer"])

I'm unable to directly answer the question since the document doesn't provide those details and instead focuses on 2022 and 2023. 


In [16]:
"""
Task 5. Advanced: Modifying the default LangChain prompt
"""

'\nTask 5. Advanced: Modifying the default LangChain prompt\n'

In [17]:
qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True
)

print(qa.combine_documents_chain.llm_chain.prompt.template)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:


In [18]:
prompt_template = """Use the context to answer the question at the end.
You must always use the context and context only to answer the question. Never try to make up an answer. If the context is empty or you do not know the answer, just say "I don't know".
The answer should consist of only 1 word and not a sentence.

Context: {context}

Question: {question}
Helpful Answer:
"""
prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
qa_chain = RetrievalQA.from_llm(
    llm=llm, prompt=prompt, retriever=retriever, return_source_documents=True
)

In [19]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.template)

Use the context to answer the question at the end.
You must always use the context and context only to answer the question. Never try to make up an answer. If the context is empty or you do not know the answer, just say "I don't know".
The answer should consist of only 1 word and not a sentence.

Context: {context}

Question: {question}
Helpful Answer:



In [20]:
search_query = "Were 2020 EMEA revenues higher than 2020 APAC revenues?"

results = qa_chain.invoke(search_query)

print("*" * 79)
print(results["result"])
print("*" * 79)
for doc in results["source_documents"]:
    print("-" * 79)
    print(doc.page_content)

*******************************************************************************
No
*******************************************************************************
-------------------------------------------------------------------------------
Year Ended December 31, % Change from 2020 2021 Prior Year EMEA revenues $ $ 43% EMEA constant currency revenues 38 % APAC revenues 32550 42% APAC constant currency revenues 40% Other Americas revenues 14404 53% Other Americas constant currency revenues 52% United States revenues 85014 39% Hedging gains (losses) 149 Total revenues $ $ 41% Revenues, excluding hedging effect $ 182351 $ Exchange rate effect (3330) Total constant currency revenues $ 254158 39% EMEA revenue growth from 2020 to 2021 was favorably affected by foreign currency exchange rates, primarily due to the US dollar weakening relative to the Euro and British pound.
-------------------------------------------------------------------------------
Google Cloud&#39;s infrastructure and 